In [1]:
#!/usr/bin/env python
# coding: utf-8
get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import time
import numpy as np
from datetime import datetime
from sklearn.externals import joblib 
import os
from konlpy.tag import Mecab
import lightgbm as lgb
print(lgb.__version__)

from sklearn import metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.externals import joblib 

import gc
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings(action='ignore')

/home/aiden/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


2.3.2


In [2]:
print(os.getcwd())

base_path = '.'

df_train = pd.read_csv(os.path.join(base_path , 'input/train.csv'), index_col=0)
df_test = pd.read_csv(os.path.join(base_path , 'input/public_test.csv'), index_col=0)
df_test['smishing'] = -1

df_fea = pd.concat([df_train, df_test])
df_fea.shape

/home/aiden/src/dacon_14


(297571, 3)

### Mecab

In [3]:
# mecab = Mecab()
# df_fea['morphs'] = df_fea['text'].apply(lambda x: mecab.morphs(x))
# df_fea['morphs_str'] = df_fea['morphs'].apply(lambda x: ' '.join(x))
# df_fea['nouns'] = df_fea['text'].apply(lambda x: mecab.nouns(x))
# df_fea['nouns_str'] = df_fea['nouns'].apply(lambda x: ' '.join(x))

# def pos(row):
#     x = row['text']
#     pos_dict = {c:0 for c in pos_cols}
    
#     for _, p in mecab.pos(x):
#         for v in p.split('+'):
#             pos_dict[v] += 1
        
#     return [pos_dict[k] for k in sorted(pos_dict.keys())]

# pos_cols = [
#     'EC',
#     'EF',
#     'EP',
#     'ETM',
#     'ETN',
#     'IC',
#     'JC',
#     'JKB',
#     'JKC',
#     'JKG',
#     'JKO',
#     'JKQ',
#     'JKS',
#     'JKV',
#     'JX',
#     'MAG',
#     'MAJ',
#     'MM',
#     'NA',
#     'NNB',
#     'NNBC',
#     'NNG',
#     'NNP',
#     'NP',
#     'NR',
#     'SC',
#     'SF',
#     'SL',
#     'SN',
#     'SSC',
#     'SSO',
#     'SY',
#     'UNA',
#     'UNKNOWN',
#     'VA',
#     'VCN',
#     'VCP',
#     'VV',
#     'VX',
#     'XPN',
#     'XR',
#     'XSA',
#     'XSN',
#     'XSV',
# ]
# df_fea[pos_cols] = df_fea.apply(pos, axis=1, result_type='expand')
# df_fea[pos_cols] = df_fea[pos_cols].astype(np.int16)
# df_fea.rename(columns={c:'fea__'+c for c in pos_cols}, inplace=True)
# df_fea.to_pickle('data/df_fea_1.pkl')

In [4]:
df_fea = pd.read_pickle('data/df_fea_1.pkl')
df_fea.head(10)

,year_month,text,smishing,morphs,morphs_str,nouns,nouns_str,fea__EC,fea__EF,fea__EP,...,fea__VA,fea__VCN,fea__VCP,fea__VV,fea__VX,fea__XPN,fea__XR,fea__XSA,fea__XSN,fea__XSV
id,,,,,,,,,,,,,,,,,,,,,
0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0,"[XXX, 은, 행성, 산, XXX, 팀장, 입니다, ., 행복, 한, 주말, 되,...",XXX 은 행성 산 XXX 팀장 입니다 . 행복 한 주말 되 세요,"[행성, 산, 팀장, 행복, 주말]",행성 산 팀장 행복 주말,0,2,1,...,0,0,1,0,0,0,0,1,0,1
1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0,"[오늘, 도, 많이, 웃, 으시, 는, 하루, 시작, 하, 세요, XXX, 은행, ...",오늘 도 많이 웃 으시 는 하루 시작 하 세요 XXX 은행 진월동 VIP 라운지 X...,"[오늘, 하루, 시작, 은행, 진월동, 라운지]",오늘 하루 시작 은행 진월동 라운지,1,1,2,...,0,0,0,2,0,0,0,0,0,1
2,2017-01,안녕하십니까 고객님. XXX은행입니다.금일 납부하셔야 할 금액은 153600원 입니...,0,"[안녕, 하, 십니까, 고객, 님, ., XXX, 은행, 입니다, ., 금일, 납부...",안녕 하 십니까 고객 님 . XXX 은행 입니다 . 금일 납부 하 셔야 할 금액 은...,"[안녕, 고객, 은행, 금일, 납부, 금액, 원, 감사, 새해, 복, 은행, 옥포]",안녕 고객 은행 금일 납부 금액 원 감사 새해 복 은행 옥포,2,5,4,...,0,0,2,2,1,0,0,0,1,3
4,2017-01,XXX 고객님안녕하세요XXX은행 XXX지점입니다지난 한 해 동안 저희 XXX지점에 ...,0,"[XXX, 고객, 님, 안녕, 하, 세요, XXX, 은행, XXX, 지점, 입니다,...",XXX 고객 님 안녕 하 세요 XXX 은행 XXX 지점 입니다 지난 한 해 동안 저...,"[고객, 안녕, 은행, 지점, 해, 동안, 저희, 지점, 성원, 감사, 시작, 년,...",고객 안녕 은행 지점 해 동안 저희 지점 성원 감사 시작 년 소망 일 고객 가정 건...,1,5,5,...,0,0,1,7,1,0,0,2,2,4
5,2017-01,1월은 새로움이 가득XXX입니다.올 한해 더 많이행복한 한해되시길바랍니다,0,"[1, 월, 은, 새로움, 이, 가득, XXX, 입니다, ., 올, 한, 해, 더,...",1 월 은 새로움 이 가득 XXX 입니다 . 올 한 해 더 많이 행복 한 한 해 되...,"[월, 한, 행복, 해]",월 한 행복 해,2,1,1,...,1,0,1,1,0,0,0,1,0,2
6,2017-01,행복한주말보내세요XXX용현남전담직원대리 XXX올림,0,"[행복, 한, 주말, 보내, 세요, XXX, 용, 현남, 전담, 직원, 대리, XX...",행복 한 주말 보내 세요 XXX 용 현남 전담 직원 대리 XXX 올림,"[행복, 주말, 현남, 전담, 직원, 대리]",행복 주말 현남 전담 직원 대리,1,1,1,...,0,0,0,2,0,0,0,1,1,0
7,2017-01,XXX 고객님 안녕하세요XXX은행 무교지점 XXX과장입니다 오늘 아침에 눈을 뜨니 ...,0,"[XXX, 고객, 님, 안녕, 하, 세요, XXX, 은행, 무교, 지점, XXX, ...",XXX 고객 님 안녕 하 세요 XXX 은행 무교 지점 XXX 과장 입니다 오늘 아침...,"[고객, 안녕, 은행, 무교, 지점, 과장, 아침, 눈, 눈, 세상, 적, 눈, 눈...",고객 안녕 은행 무교 지점 과장 아침 눈 눈 세상 적 눈 눈 순간 출근 걱정 어른 ...,15,8,10,...,5,0,2,22,2,0,2,2,2,4
8,2017-01,XXX 고객님지난 한해 베풀어 주신 은혜 진심으로 감사 드립니다.가슴 깊이 간직 하...,0,"[XXX, 고객, 님, 지난, 한, 해, 베풀, 어, 주, 신, 은혜, 진심, 으로...",XXX 고객 님 지난 한 해 베풀 어 주 신 은혜 진심 으로 감사 드립니다 . 가슴...,"[고객, 한, 은혜, 진심, 감사, 가슴, 간직, 정유, 년, 새해, 가족, 행복,...",고객 한 은혜 진심 감사 가슴 간직 정유 년 새해 가족 행복 뜻 바 진심 소망 은행...,3,3,4,...,0,0,0,5,1,0,0,2,1,2
9,2017-01,설연휴 가족들과 훈훈한 정 나누시고 정겨운추억 많이 만드세요XXX오XXX올림,0,"[설, 연휴, 가족, 들, 과, 훈훈, 한, 정, 나누, 시, 고, 정겨운, 추억,...",설 연휴 가족 들 과 훈훈 한 정 나누 시 고 정겨운 추억 많이 만드세요 XXX 오...,"[설, 연휴, 가족, 정, 추억, 오]",설 연휴 가족 정 추억 오,3,0,2,...,1,0,0,3,0,0,1,1,1,0


In [5]:
df_fea.columns

Index(['year_month', 'text', 'smishing', 'morphs', 'morphs_str', 'nouns',
       'nouns_str', 'fea__EC', 'fea__EF', 'fea__EP', 'fea__ETM', 'fea__ETN',
       'fea__IC', 'fea__JC', 'fea__JKB', 'fea__JKC', 'fea__JKG', 'fea__JKO',
       'fea__JKQ', 'fea__JKS', 'fea__JKV', 'fea__JX', 'fea__MAG', 'fea__MAJ',
       'fea__MM', 'fea__NA', 'fea__NNB', 'fea__NNBC', 'fea__NNG', 'fea__NNP',
       'fea__NP', 'fea__NR', 'fea__SC', 'fea__SF', 'fea__SL', 'fea__SN',
       'fea__SSC', 'fea__SSO', 'fea__SY', 'fea__UNA', 'fea__UNKNOWN',
       'fea__VA', 'fea__VCN', 'fea__VCP', 'fea__VV', 'fea__VX', 'fea__XPN',
       'fea__XR', 'fea__XSA', 'fea__XSN', 'fea__XSV'],
      dtype='object')

### Diff Dict

In [9]:
vocab_neg_set = set()
_ = df_fea[df_fea['smishing'] == 0]['morphs'].apply(lambda x: [vocab_neg_set.add(c) for c in x])
len(vocab_neg_set)

47094

In [10]:
vocab_pos_set = set()
_ = df_fea[df_fea['smishing'] == 1]['morphs'].apply(lambda x: [vocab_pos_set.add(c) for c in x])
len(vocab_pos_set)

6205

In [11]:
vocab_test_set = set()
_ = df_fea[df_fea['smishing'] == -1]['morphs'].apply(lambda x: [vocab_test_set.add(c) for c in x])
len(vocab_test_set)

7760

In [20]:
pos_size = (df_fea['smishing'] == 1).sum()
neg_size = (df_fea['smishing'] == 0).sum()
pos_size, neg_size

(18703, 277242)

In [19]:
vocab_neg_set & vocab_pos_set

4498

In [41]:
from collections import Counter


In [81]:
total = []
_ = df_fea[df_fea['smishing'] == 0]['morphs'].apply(lambda x: total.extend(x))

df_neg = pd.DataFrame.from_dict(Counter(total),orient='index', columns=['neg_cnt'])
df_neg['neg_rate'] = df_neg['neg_cnt'] / neg_size

In [82]:
total = []
_ = df_fea[df_fea['smishing'] == 1]['morphs'].apply(lambda x: total.extend(x))

df_pos = pd.DataFrame.from_dict(Counter(total),orient='index', columns=['pos_cnt'])
df_pos['pos_rate'] = df_pos['pos_cnt'] / pos_size

In [83]:
df = pd.merge(df_pos, df_neg, how='outer', left_index=True, right_index=True)

In [84]:
df['diff'] = (df['pos_rate'] - df['neg_rate']).abs()

In [85]:
df_res = df.sort_values('diff', ascending=False).reset_index()

In [91]:
df_res[(df_res['index'].str.len() > 1) & (df_res['diff'] > 1)]['index'].values

array(['가능', '상품', '금리', '으로', '등급', '대출', 'XXX', '상담', '습니다', '신용', '한도',
       '신청', '까지', '드립니다', '부채', '진행', '통합', '드리', '문자', '은행', '거부', '이상',
       '직장', '추가', '전환', '사용', '방식', '합니다', '방법', '광고', '전화', '자격', '수신',
       '이자', '에서', '대환', '자금', '거나', '금융'], dtype=object)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(#analyzer='char',
                              vocabulary=vocab_neg_set & vocab_pos_set,
                             stop_words=[], 
#                              max_df=1.0,
#                              min_df=100
)

vectorizer = vectorizer.fit(df_fea['morphs_str'].values)

In [22]:
vectorizer.get_feature_names()

['!',
 '!!',
 '!!!',
 '!!(',
 '!!-',
 '!(',
 '!)',
 '!-',
 '!.',
 '%',
 '%!',
 '%(',
 '%)',
 '%)(',
 '%)-',
 '%-',
 '(',
 ')',
 '-',
 '-!',
 '-(',
 '--',
 '---',
 '-----',
 '------------',
 '-------------',
 '--------------',
 '---------------',
 '---------------------',
 '-------------------------',
 '-?',
 '.',
 '.(',
 '..',
 '..(',
 '..-',
 '...',
 '....-',
 '0',
 '00',
 '000',
 '0000',
 '001',
 '0011',
 '0018',
 '002',
 '004',
 '005',
 '01',
 '010',
 '0102',
 '0103',
 '0104',
 '0105',
 '0109',
 '012',
 '013',
 '015',
 '017',
 '02',
 '023',
 '024',
 '03',
 '031',
 '04',
 '05',
 '055',
 '06',
 '07',
 '070',
 '0709',
 '072',
 '0727',
 '08',
 '080',
 '0808',
 '09',
 '0926',
 '097',
 '0976',
 '0988',
 '1',
 '10',
 '100',
 '1000',
 '10000',
 '1001',
 '1002',
 '1003',
 '1004',
 '1005',
 '1006',
 '101',
 '1013',
 '1015',
 '1016',
 '1019',
 '102',
 '1020',
 '104',
 '105',
 '10530',
 '1054',
 '106',
 '108',
 '1080',
 '11',
 '110',
 '1100',
 '11000',
 '111',
 '1111',
 '1112',
 '11128',
 '1114

In [ ]:
cnt_vec = vectorizer.transform(df_fea['text'].values).toarray()

cnt_dict = {'cnt_{0:04d}'.format(i):'cnt_{0:04}_{1}'.format(i, c) for i, c in enumerate(vectorizer.get_feature_names())}
fea_dict.update(cnt_dict)
cnt_cols = sorted(cnt_dict.keys())

df_cnt_vec = pd.DataFrame(data=cnt_vec, index=df_fea.index, columns=cnt_cols, dtype=np.float16)
df_cnt_vec = df_cnt_vec.loc[:, (df_cnt_vec != 0).any(axis=0)]
dfs.append(df_cnt_vec)
df_cnt_vec.shape

In [14]:
len((vocab_neg_set | vocab_pos_set) & vocab_test_set)

6582

In [15]:
len(vocab_pos_set & vocab_test_set)

2958

In [17]:
len(vocab_test_set - (vocab_neg_set | vocab_pos_set))
vocab_test_set - (vocab_neg_set | vocab_pos_set)

{'국무총리실',
 'krzzknF',
 '1655',
 '대삼',
 '887520',
 '000940',
 'kbg',
 '유오',
 '2056',
 '돋워',
 '총궐기',
 '후성',
 '클렌',
 'google',
 '1435',
 '정판',
 '뉴트',
 'siawasekbfg',
 '500473',
 '86737',
 '신불자',
 '26667',
 '장년층',
 'EYPFWeIvNMpPYlZ',
 'evtpage',
 '임일',
 'JHfg',
 '최석태',
 '8588',
 'KTX',
 'bitly',
 '간선',
 '49752',
 '당산',
 '대조표',
 'se',
 'BNKXXX',
 '39760',
 '슬라이스',
 'ㅇ경품',
 'xmlnshttp',
 '초임',
 'ParameterXXX',
 'mbank',
 '27990',
 'HPxdNuyQj',
 'GrPHAa',
 '블랙박스',
 '파헤치',
 '미신고',
 '대야',
 '가브리엘',
 'pF',
 '깐깐',
 '른주말모든짐벗어두고여유를즐기시길빕니다',
 'krMWMO',
 'XXXnbXXX',
 'BGlIvW',
 '입질',
 '움보',
 '크래비스',
 'productCdDP',
 '신용한',
 '050774',
 'Credit',
 '회리바람꽃',
 'BY',
 '욕실',
 '테크니컬',
 '묶이',
 '총파업',
 '위자',
 '601662',
 '첼시',
 '27879',
 '현물출자',
 '41860',
 '구청장',
 'rxuQel',
 '사간',
 '2692',
 '석하',
 '019478',
 '0002',
 '경비대',
 '등하',
 'BA',
 '051343',
 '192020',
 'blog',
 '군포역',
 'ㅇ전담창구운영',
 '선언문',
 '클렌징',
 '캡',
 '161929',
 'BD',
 '후생',
 '용스',
 '2980',
 '말레이시아',
 '상임',
 '셰프',
 '0021',
 '명암',
 'nonghyup',
 '피부과',
 '

### Seq Fea

#### MAX_COUNT

In [ ]:
def toidx(src_col, max_len, reverse=False):
    idx_col_nm = '{}_{}_idx'.format(src_col, max_len)
    if reverse:
        idx_col_nm = idx_col_nm.replace('idx', 'ridx')

    vocab_set = set()
    _ = df_fea[src_col].apply(lambda x: [vocab_set.add(c) for c in x])
    
    vocab_dict = {v: i+1 for i, v in enumerate(vocab_set)}
    vocab_dim = len(vocab_dict.keys()) + 1

    def _toidx(x):
        if reverse:
            x = x[::-1]
        return [int(vocab_dict[x[i]]) if i < len(x) else 0 for i in range(max_len)]

#     print(vocab_dim, max_len)
    
    return idx_col_nm, df_fea[src_col].apply(_toidx), vocab_dim, max_len

In [ ]:
src_col = 'morphs'
# src_col = 'text'
max_len = 128#512#256

In [ ]:
# from sklearn.feature_extraction.text import CountVectorizer

# vectorizer = CountVectorizer(max_features=1000)
# vectorizer = vectorizer.fit(df_fea['morphs_str'].values)

# seq_vocab = set(vectorizer.get_feature_names())

# org_max_len = df_fea[src_col].str.len().max()
# df_fea[src_col] = df_fea[src_col].apply(lambda x: [m for m in x if m in seq_vocab])
# max_len = df_fea[src_col].str.len().max()
# print(org_max_len, 'to', max_len)

#### From 0

In [ ]:
c, d, vocab_dim, max_len = toidx(src_col, max_len)
df_fea[c] = d
c, vocab_dim, max_len

#### From -1

In [ ]:
c, d, vocab_dim, max_len = toidx(src_col, max_len, reverse=True)
df_fea[c] = d
c, vocab_dim, max_len

### Basic Fea

In [ ]:
df_fea['fea__text_len'] = df_fea['text'].str.len().fillna(0).astype(np.float16)
df_fea['fea__morphs_cnt'] = df_fea['morphs'].apply(lambda x: len(x)).fillna(0).astype(np.float16)
# df_fea['fea__noun_cnt'] = df_fea['nouns'].apply(lambda x: len(x)).fillna(0).astype(np.float16)

for c in [c for df_fea.columns if 'fea__' in c]:
    df_fea[f'rate_{c}_textlen'] = df_fea[c]  /  df_fea['fea__text_len']
    df_fea[f'rate_{c}_morphcnt'] = df_fea[c]  /  df_fea['fea__morphs_cnt']
    
df_fea

### Count

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(analyzer='char',
#                              vocabulary=vocab,
                             stop_words=stop_words, 
                             max_df=1.0,
                             min_df=100)

# vectorizer = vectorizer.fit(df_fea[df_fea['smishing']==1]['nouns_str'].values)
vectorizer = vectorizer.fit(df_fea['text'].values)
cnt_vec = vectorizer.transform(df_fea['text'].values).toarray()

cnt_dict = {'cnt_{0:04d}'.format(i):'cnt_{0:04}_{1}'.format(i, c) for i, c in enumerate(vectorizer.get_feature_names())}
fea_dict.update(cnt_dict)
cnt_cols = sorted(cnt_dict.keys())

df_cnt_vec = pd.DataFrame(data=cnt_vec, index=df_fea.index, columns=cnt_cols, dtype=np.float16)
df_cnt_vec = df_cnt_vec.loc[:, (df_cnt_vec != 0).any(axis=0)]
dfs.append(df_cnt_vec)
df_cnt_vec.shape

### tfidf

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

def tfidf(data, params, tfidf_tag):
    vectorizer = TfidfVectorizer(**params)
    vectorizer = vectorizer.fit(data)

    d = {'{0}_{1:04d}'.format(tfidf_tag, v):'{0}_{1:04d}_{2}'.format(tfidf_tag, v, k) for k, v in sorted(vectorizer.vocabulary_.items(), key=lambda item: item[1])}
    c = sorted(d.keys())

    df = pd.DataFrame(
#         data=vectorizer.transform(df_fea['nouns_str'].values).toarray(),
        data=vectorizer.transform(df_fea['morphs_str'].values).toarray(),
        columns=c, 
        index=df_fea.index,
        dtype=np.float16)
    
    # Remove all zeros column
    df = df.loc[:, (df != 0).any(axis=0)]
    d = {k:v for k, v in d.items() if k in df.columns}
    
    print(tfidf_tag, df.shape)
    
    return df, d, vectorizer

In [ ]:
tfidf_src_col = 'morphs_str'

#### 1, 1

##### char

In [ ]:
params = {
    'analyzer' : 'char',
    'max_features':None, 
#     'vocabulary': vocab,
    'stop_words':stop_words, 
#     'max_df':0.1, 
    'min_df':100, 
    'ngram_range':(1, 1), 
}
df, d, v = tfidf(df_fea[tfidf_src_col].values, params, 'tfidf_char_11')
dfs.append(df)
fea_dict.update(d)

##### word

In [ ]:
params = {
    'max_features': 3000,#2500,#2000, 
#     'vocabulary': vocab,
    'stop_words':stop_words, 
    'max_df':1.0, 
    'min_df':200, 
    'ngram_range':(1, 1), 
}
df, d, v = tfidf(df_fea[tfidf_src_col].values, params, 'tfidf_word_11')
dfs.append(df)
fea_dict.update(d)

#### 2, 2

In [ ]:
params = {
    'max_features':3000,#2500,#2000, 
#     'vocabulary': vocab,
    'stop_words':stop_words, 
    'max_df':1.0, 
    'min_df':500, 
    'ngram_range':(2, 2), 
}
df, d, v = tfidf(df_fea[tfidf_src_col].values, params, 'tfidf_word_22')
dfs.append(df)
fea_dict.update(d)

#### 3, 3

In [ ]:
params = {
    'max_features':1500, #1000,#500, 
#     'vocabulary': vocab,
    'stop_words':stop_words, 
    'max_df':1.0, 
    'min_df':500, 
    'ngram_range':(3, 3), 
}
df, d, v = tfidf(df_fea[tfidf_src_col].values, params, 'tfidf_word_33')
dfs.append(df)
fea_dict.update(d)

#### smishing 1, 1

##### char

In [ ]:
params = {
    'analyzer' : 'char',
#     'max_features':500, 
#     'vocabulary': vocab,
    'stop_words':stop_words, 
#     'max_df':1.0, 
#     'min_df':100, 
    'ngram_range':(1, 1), 
}
df, d, v = tfidf(df_fea[df_fea['smishing']==1][tfidf_src_col].values, params, 'tfidf_pos_char_11')
dfs.append(df)
fea_dict.update(d)


##### word

In [ ]:
params = {
    'max_features':1000,#500, 
#     'vocabulary': vocab,
    'stop_words':stop_words, 
    'max_df':1.0, 
    'min_df':100, 
    'ngram_range':(1, 1), 
}
df, d, v = tfidf(df_fea[df_fea['smishing']==1][tfidf_src_col].values, params, 'tfidf_pos_word_11')
dfs.append(df)
fea_dict.update(d)


#### smishing 2, 2

In [ ]:
params = {
    'max_features':1000,#,500, 
#     'vocabulary': vocab,
    'stop_words':stop_words, 
    'max_df':1.0, 
    'min_df':100, 
    'ngram_range':(2, 2), 
}
df, d, v = tfidf(df_fea[df_fea['smishing']==1][tfidf_src_col].values, params, 'tfidf_pos_word_22')
dfs.append(df)
fea_dict.update(d)


### W2V

In [ ]:
# ts = '20191214T055747'
# file_name = 'model/gensim_{}'.format(ts)

# w2v_model = joblib.load(os.path.join(base_path, '{}.pkl'.format(file_name)))
# w2v_size = w2v_model.wv.vectors.shape[1]

# def mean_w2v(row):
#     nouns = row['nouns']
#     w2v = np.zeros(w2v_size)

    
#     for n in nouns:
#         if n in w2v_model.wv.vocab.keys():
#             w2v = np.add(w2v, w2v_model.wv[n])
            
#     return w2v if len(nouns) == 0 else np.true_divide(w2v, len(nouns))

# w2v_cols = ['w2v_{}'.format(i) for i in range(w2v_size)]

# df_fea[w2v_cols] = df_fea.apply(mean_w2v, axis=1, result_type='expand')

# for c in w2v_cols:
#     df_fea[c] = df_fea[c].astype(np.float16)

### Merged

In [ ]:
print(df_fea.shape)
for df in dfs:
    print(df.shape)

df_merged = pd.concat([df_fea] + dfs, axis=1)
print('df_merged', df_merged.shape)
print(df_merged.info())
# df_merged.head(10)


In [ ]:
idx_cols = ['id', 'year_month', 'text', 'smishing', 'nouns', 'nouns_str', 'morphs', 'morphs_str']

cat_cols = []
fea_cols = [c for c in df_merged.columns if c not in idx_cols]

for c in fea_cols:
    if c not in fea_dict.keys():
        fea_dict[c] = c

len(fea_cols), len(fea_dict.keys()), vocab_dim, max_len

#### MinMaxScaler

In [ ]:
del df_fea, df_train, df_test
for df in dfs:
    del df
gc.collect()

In [ ]:
# base_dict = (df_merged[fea_cols].max() - df_merged[fea_cols].min()).to_dict()

# for c in fea_cols:
#     df_merged[c] = df_merged[c] / base_dict[c]

# print(df_merged[fea_cols].info())

In [ ]:
# merged_ts = datetime.now().strftime('%Y%m%dT%H%M%S') + '_' + str(len(fea_cols))
merged_ts = '{}_{}_{}_{}'.format(datetime.now().strftime('%Y%m%dT%H%M%S'), 
                                 str(len(fea_cols)), 
                                 str(max_len), 
                                 str(vocab_dim))
print(merged_ts)

for c in df_merged:
    if c not in fea_cols + ['smishing']:
        df_merged.drop(c, axis=1, inplace=True)
        

In [ ]:
df_merged.info()

### Dump

In [ ]:
joblib.dump(fea_dict, 'data/df_merged_{}_fea_dict.pkl'.format(merged_ts))

In [ ]:
joblib.dump(df_merged.loc[df_merged['smishing'] != -1,:], 'data/df_merged_{}_train.pkl'.format(merged_ts))

In [ ]:
joblib.dump(df_merged.loc[df_merged['smishing'] == 1,:], 'data/df_merged_{}_train_pos.pkl'.format(merged_ts))

In [ ]:
joblib.dump(df_merged.loc[df_merged['smishing'] == -1,:], 'data/df_merged_{}_test.pkl'.format(merged_ts))

In [ ]:
# del df_merged, df_fea, df_train, df_test
# for df in dfs:
#     del df

In [ ]:
gc.collect()

In [ ]:
exit(0)